In [1]:
# Skriptas mit2zive_creat_beats_attrib skirtas mit2zive EKG įrašuose esančių pūpsnių atributų sukūrimui, kad būtų
# galima juos panaudoti sekų formavimui. Į atributus įrašomi ir RR intervalai: RRl, RRr
# Atributų eilutėse, atitinkančios pirmą ir paskutinį pūpsnį, RRl ir RRr reikšmės lygios sentinel = -1   

# Į aplanką rec_dir patalpinams failas info_zirkles.json, 
# failas all_beats_attr.csv su pūpsnių: {userNr, recordingNr, userId, recordingId, sample, symbol},
# 
# 
# //////////////////////////////////////////////////////////////////////////////////////////////////////
#

import pandas as pd
import numpy as np
from numpy import loadtxt, savetxt
import matplotlib.pyplot as plt
from pathlib import Path
from icecream import ic
import json, time, sys

from mit2zive_util import get_seq_start_end, create_dir
from mit2zive_util import anotaciju_pasiskirstymas_v2, runtime

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'ZIVE&MIT_TST'

# Aplankas su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_folder = 'records'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Failai pūpsnių klasių formavimui
annot_grouping = {'N':'N','R':'N', 'L':'N', 'e':'N', 'j':'N', 'A':'S','a':'S', 'J':'S', 'S':'S', 'V':'V', 'E':'V'}
selected_beats = {'N':0, 'S':1, 'V':2}

# ///////////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
subj_list =  info_create['subj_list'] # pacientų įrašų sąrašas
annot_list  = info_create['annot_list'] # anotacijų sąrašas
# print(annot_list)

# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(13))
beats_annot = dict(zip(annot_list, nr_sequence))

print("\nSkriptas mit2zive_prep skirtas EKG įrašų pjaustymui į užduoto ilgio sekas")
print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", db_folder)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų sąrašas:\n",subj_list)
print("\nAnotacijų sąrašas:\n", annot_list)


OS in my system :  win32

Skriptas mit2zive_prep skirtas EKG įrašų pjaustymui į užduoto ilgio sekas

Bendras Zive duomenų aplankas:  C:\DI\Data\MIT&ZIVE
Zive EKG įrašų aplankas:  ZIVE&MIT_TST
Diskretizavimo dažnis:  200

Pacientų įrašų sąrašas:
 ['124', '209', '228']

Anotacijų sąrašas:
 ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', 'Q']


In [2]:
# I-a dalis. Nustatomas anotacijų pasiskirstymas per visus įrašus.

# Atliekama pirminė EKG įrašų anotacijų analizė

# Pasiruošimas ciklui per pacientų įrašus
labels_rec_all = pd.DataFrame(columns=beats_annot.keys(),dtype=int)
labels_rec_all.insert(0,"subject",0)
labels_rec = []

# Pacientų įrašų sąrašas bandymams (yra dar viena vieta):
# subj_list = ['100','101', '230', '231']

# Ciklas per pacientų įrašus

for subject in subj_list:
    # Paciento anotacijų nuskaitymas, nustatomas anotacijų 
    # ir užduotų klasių pasiskirstymas per visus įrašus

    labels_rec = np.zeros(labels_rec_all.shape[1],dtype=int)
    labels_rec[0] = subject

    # Nuskaitome anotacijas ir jų indeksus
    file_path = Path(rec_dir, subject + '.json')
    df = pd.read_json(file_path, orient = 'records')
    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol = df['annotationValue'].to_numpy()

    # Ciklas per visas paciento įrašo anotacijas (simbolius)
    for symbol in atr_symbol:
        # Gaunamas anotacijos simbolio numeris anotacijų sąraše
        label = beats_annot.get(symbol)
        if (label == None):
            continue
        labels_rec[label+1] +=1

    # Sumuojame į bendrą masyvą
    # print(labels_rec)
    labels_rec_all.loc[len(labels_rec_all)] = labels_rec

# Ciklo per pacientų įrašus pabaiga

# Parodomas anotacijų ir užduotų klasių pasiskirstymas per visus įrašus
print("\nAnotacijų pasiskirstymas per visus įrašus\n")
print("labels_rec_all:\n")
print(labels_rec_all)
suma = labels_rec_all.iloc[:,1:].sum()
print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()
print(' '*5, 'total: ', total)


Anotacijų pasiskirstymas per visus įrašus

labels_rec_all:

   subject     N  L     R  e  j    A  a   J  S    V  E  F   Q
0      124     0  0  1531  0  5    2  0  29  0   47  0  5  15
1      209  2621  0     0  0  0  383  0   0  0    1  0  0  47
2      228  1688  0     0  0  0    3  0   0  0  362  0  0  88

       sum:  4309, 0, 1531, 0, 5, 388, 0, 29, 0, 410, 0, 5, 150
      total:  6827


In [3]:
# II-a dalis. mit2zive EKG įrašai analizuojami, formuojamas freimas all_beats_attr ir  įrašomas į diską. 
print("\nEKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską")

# Pacientų įrašų sąrašas bandymams:
# subj_list = ['100','101', '230', '231']


# Sukūriame masyvą sekų atributų sąrašo kaupimui
all_beats_attr = pd.DataFrame(columns= ['userNr', 'recordingNr', 'sample', 'symbol'])

start_time = time.time()

# CIKLAS PER PACIENTŲ ĮRAŠUS

for subject in subj_list:
    print("\nPacientas:", subject)
    
    # Paciento anotacijų ir EKG įrašų nuskaitymas, sekų išpjovimas ir įrašymas, sekų atributų formavimas

    # Nuskaitome paciento anotacijas ir jų indeksus
    file_path = Path(rec_dir, subject + '.json')
    df = pd.read_json(file_path, orient = 'records')
    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol = df['annotationValue'].to_numpy()
    
    subject_labels = []
    beat_nr = 0
    icycle = 0

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):
    
        icycle +=1
        if (icycle%show_period == 0):
            print(icycle, end =' ') 

        # Formuojame pūpsnio atributus
        beats_attr = {'userNr':int(subject), 'recordingNr':0, 'sample':i_sample, 'symbol':atr_symbol[i]}
        all_beats_attr = all_beats_attr.append(beats_attr, ignore_index=True)

        beat_nr +=1

# Ciklo per pacientų įrašus pabaiga

# Konvertuojame anotacijų atributų masyvą -  atsikratome 'Q', 'F'
# ir sustambiname anotacijas iki makro anotacijų 'N', 'S', 'V'

# Atsikratome nepageidaujamų anotacijų: 'Q'
index_names = all_beats_attr[all_beats_attr['symbol'].isin(['Q'])].index
all_beats_attr.drop(index_names, inplace = True)
# print(all_beats_attr.info())

# Atsikratome nepageidaujamų anotacijų: 'F'
index_names = all_beats_attr[all_beats_attr['symbol'].isin(['F'])].index
all_beats_attr.drop(index_names, inplace = True)

# Anotacijas stambinam į makro anotacijas
# thickening_mapping_dict = {'L': 'N', 'R': 'N','e':'N', 'j': 'N', 'A': 'S', 'a': 'S', 'J': 'S', 'E':'V'}
all_beats_attr["symbol"].replace(annot_grouping, inplace=True)
# all_beats_attr["symbol"].replace(thickening_mapping_dict, inplace=True)
# https://www.geeksforgeeks.org/python-pandas-dataframe-replace/
# print(all_beats_attr)

# Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
all_beats_attr.reset_index(inplace = True, drop = True)

# Iš anotacijų padarome klasių numerius ir pridedame, kaip naują stulpelį
annot_labels = {key:selected_beats[value] for key, value in annot_grouping.items()}
labels_from_annot = all_beats_attr['symbol'].replace(annot_labels, inplace=False)
all_beats_attr['label'] = labels_from_annot

print("\n")
print("\nVisi duomenys panaikinus 'Q', 'F' ir po grupavimo į makro anotacijas: \n", annot_grouping,"\n")
labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, cols_pattern=annot_list)
print(labels_table)
print("\nTotal:", labels_sums["All"])

# Suformuojame stulpelius "RRl" ir "RRr" ir pridedame, kaip naujus stulpelius
# Atributų eilutėse, atitinkančios pirmą ir paskutinį pūpsnį, RRl ir RRr reikšmės lygios sentinel = -1

# Suformuojame RR intervalų poras apie kiekvieną R dantelį
grouped = all_beats_attr.groupby('userNr')
RR_array = np.empty((0,2), int)

for userNr in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin
    # print(f'\nGroup: {key}\n{df_seq_errors.loc[grouped.groups[key]]}')
    samples = all_beats_attr.loc[grouped.groups[userNr]]['sample'].to_numpy(dtype=int)
    len_samples = len(samples)
    RR_array_user = np.zeros(shape=(len_samples,2), dtype=int)
    for idx in range(1, len_samples-1):
        RR_array_user[idx,0] = samples[idx] - samples[idx-1]
        RR_array_user[idx,1] = samples[idx+1] - samples[idx]
    RR_array_user[0,0] = -1
    RR_array_user[0,1] = -1
    RR_array_user[len_samples-1,0] = -1    
    RR_array_user[len_samples-1,1] = -1
    RR_array = np.vstack((RR_array,RR_array_user))
all_beats_attr = pd.concat([all_beats_attr, pd.DataFrame(RR_array)], axis=1)
all_beats_attr.columns = ['userNr', 'recordingNr', 'sample', 'symbol', 'label', 'RRl', 'RRr']

# Įrašome sekos atributų masyvą į seq_dir aplanką
file_path = Path(rec_dir, 'all_beats_attr.csv')
all_beats_attr.index.name = "id"
all_beats_attr.to_csv(file_path)
print("\nAtributų freimas įrašytas: į ", file_path, "\n" )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

print("\nPabaiga.............")



EKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską

Pacientas: 124
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 
Pacientas: 209
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 
Pacientas: 228
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 


Visi duomenys panaikinus 'Q', 'F' ir po grupavimo į makro anotacijas: 
 {'N': 'N', 'R': 'N', 'L': 'N', 'e': 'N', 'j': 'N', 'A': 'S', 'a': 'S', 'J': 'S', 'S': 'S', 'V': 'V', 'E': 'V'} 

symbol     N    S    V
userNr                
124     1536   31   47
209     2621  383    1
228     1688    3  362
All     5845  417  410

Total: 6672

Atributų freimas įrašytas: į  C:\DI\Data\MIT&ZIVE\ZIVE&MIT_TST\records\all_beats_attr.csv 



Runtime: 00:00:16

Pabaiga.............
